In [ ]:
#Make sure to run this cell as the start
from quantum import *

In [ ]:
# create a qubit using the qbit(x) function, where x is the default value of the bit:
q0 = qbit(1)
q1 = qbit(0)
#qubit 1 is represented by [0 1] while qubit 0 is [1 0]


#all qbits are just a list of 2 elements, elements being either real or complex
#complex numbers in this library are handled with `comp` class, and Matrices with `Matrix` class
print("q0:", q0, ", q1:", q1)

In [ ]:
#gates are applied on qubits using the corresponding gate funtions.
#as of now, IDEN (identity), NOT (not), HAD (hadamard), CNOT (controlled not) gates have been implemented. 
#however, after application of CNOT gate I still haven't figured out how to use other gates

#to see the outcome of a computation, use the MEASURE function
#Measurement can be done on a single qubit or a set of qubits. If multiple qubits are to be measured,
#then we must pass the inner tensor product of these qubits to MEASURE. tensor product can be 
#calculated using the tensor function.

In [ ]:
#example: measures a single qubit (here the qubit is 1, so measurement is always 1, try running this line a few times)
print(MEASURE(q0))
# print(MEASURE(q1))

#example: measuring a set of qubits:
q0andq1 = tensor(q0, q1)
print(MEASURE(q0andq1))

In [ ]:
#example: displaying measurements in a readable way
#use the extract function to get the value of a single qubit from a measurement of set of qubits
measurement = MEASURE(q0andq1)
q0m = extract(measurement, 0) #here 0 is the qbitindex as it is the 0th element in the tensor product
q1m = extract(measurement, 1)
print("q0m: ", q0m, ", q1m: ", q1m)

In [ ]:
#so far nothing of note, we created qubits 0 and 1, and measured them together and individually, and found
#the same qubits.

#this is where things get interesting

In [ ]:
#list of gates so far:
#IDEN -> identity, return the same exact value with no change
#NOT -> returns the logical not value of the input value
#HAD -> Hadamard gate, can put a qubit into superposition of take a qubit already in superposition to normalcy
#CNOT -> takes 2 qubits as input; one qubit is designated control, the other target.
#        if the control qubit is 0, nothing is done. if the control qubit is 1, the target qubit gets the
#        NOT gate applied on it.

#example: hadamard gate in action
print(HAD(q0))

In [ ]:
#example: measuring 2 qubits in independent superpositions:
q0s = HAD(q0)
q1s = HAD(q1)

measurement = MEASURE(tensor(q0s, q1s))

q0m = extract(measurement, 0)
q1m = extract(measurement, 1)

print("q0 value: ", q0m, ", q1 value: ", q1m)
#try running this cell a few times to see the various outcomes

In [ ]:
# in the previous example, the qubits were put in superposittion, meaning each had a 50-50 chance of being
# either 0 or 1. since both the bits were in superposition, the chance of seeing 00, 01, 10, 11 is 
# equally likely, even though we started with qubits that were only 0 or 1

#to automate the process we did in previous example, the run function can be used.

#example: running a measurement multiple times and viewing the outcome:
run(shots = 16000, state = tensor(q0s, q1s))

#try running the experiment a few times to see the outcomes.
#each of the possible states, represented by |Ψ  > is a possible state of the set of qubits.
# the percentages show what percent of the total number of measurements made (here 1600) were that particular state
#so in this case, where both the bits are euqally likely to be 0 or 1, 00, 01, 10, 11 are all equally likely
#hence, each has a 25% share of the total measurements

In [ ]:
#example: multiple superposition (no limit to how many can be in superpostion):
state = tensor(HAD(qbit(0)), tensor(HAD(qbit(0)), HAD(qbit(0))))
run(1600, state)

In [ ]:
#example: using CNOT:
measurement = MEASURE(CNOT(q0, q1))

q0m = extract(measurement, 0)
q1m = extract(measurement, 1)

print("q0 value: ", q0m, ", q1 value: ", q1m)

#the result is as expected (note, we are using the normal versions of the qubits in this example)

In [ ]:
#example: quantum entanglement:

q0h = HAD(q0) #one of the bits is in superposition

#the bits are now put into entanglement:
state = CNOT(q0h, q1)


run(1600, state)
#both the bits are always the same value, that is knowing the value of one tells you the other's

In [ ]:
#example: quantum entanglement 2:

q0h = HAD(q0) #one of the bits is in superposition

#the bits are now put into entanglement:
state = CNOT(q0h, qbit(1))


run(1600, state)
#both the bits are always the oppposite value, that is knowing the value of one tells you the other's

In [ ]:
#dealing with invidual qubits can get tedious, instead a quantum program or qprogram can be used
#to simplify all this stuff and run experiments quickly

In [ ]:
#example: using a qprogram:

#make a qprogram with some qubits
myqprogram = qprogram(nqbits = 2)
#qubits are always labeled q0 -> q(nqbits-1) from top to bottom
#each qubit has a gate-line, which act somewhat like a quantum circuit

#adding gates to the qprogram's lines:
myqprogram.addgates(qbitindex = 0, gates = [HGATE(), HGATE(), IGATE(), NGATE()])
myqprogram.addgates(qbitindex = 1, gates = [IGATE(), CNOTGATE()])
#two things of note:
#gates supported in qprograms as of now are:
#hadamard, identity, cnot, not (with the above gate names)
#secondly, cnot gate is assumed to have the target bit at the place it is being called, while the control
#qubit is direclty the one above it, with the control handle directly above it

#compile the program to see its circuit diagram:
myqprogram.compile()
#note that identity gates are automatically added to make it easier to understand the working of two input
#gates like cnot


In [ ]:
#run the program to see the output:

myqprogram.run()

In [ ]:
#note: the run function on a program and standalone run functions are the same, and hence take the same
#parameters. run can also graph the results
myqprogram.run(binary = True, graph = True)
#if you are using an ide, the autocomplete should prompt you with all the optional parameters of run that
#are available